In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# 1. Cargar y preparar datos
df = pd.read_csv("top20nuevo.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 2. Agrupar por producto y mes
df_prod = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# 3. Crear features adicionales

# 3.1 Promedio histórico de febreros
febreros = df_prod[df_prod['periodo'].dt.month == 2]
febrero_hist = febreros[febreros['periodo'].dt.year < 2020].groupby('product_id')['tn'].mean().reset_index()
febrero_hist.rename(columns={'tn': 'febrero_hist'}, inplace=True)

# 3.2 Media móvil 12 meses
df_prod = df_prod.sort_values(['product_id', 'periodo'])
df_prod['media_movil_12m'] = (
    df_prod.groupby('product_id')['tn']
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)

# 4. Crear lags
def crear_lags(df, lags=[1, 2, 3]):
    df_lags = df.copy()
    for lag in lags:
        df_lags[f'lag_{lag}'] = df_lags.groupby('product_id')['tn'].shift(lag)
    return df_lags

df_lags = crear_lags(df_prod)
df_lags['mes'] = df_lags['periodo'].dt.month

# 5. Calcular deltas
df_lags['var_lag1_lag2'] = df_lags['lag_1'] / df_lags['lag_2']
df_lags['delta_lag1_lag2'] = df_lags['lag_1'] - df_lags['lag_2']
df_lags['delta_rel_lag1_lag2'] = df_lags['delta_lag1_lag2'] / df_lags['lag_2']
df_lags['delta_ma12_lag1'] = df_lags['lag_1'] - df_lags['media_movil_12m']
df_lags.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

# 6. Preparar entrenamiento y validación
df_train = df_lags.dropna()

features = [
    'lag_1', 'lag_2', 'lag_3', 'mes', 'media_movil_12m', 'var_lag1_lag2',
    'delta_lag1_lag2', 'delta_rel_lag1_lag2', 'delta_ma12_lag1'
]
target = 'tn'

train_data = df_train[df_train['periodo'] < '2019-12-01']
val_data = df_train[df_train['periodo'] == '2019-12-01']

X_train = train_data[features].copy()
y_train = train_data[target]
X_val = val_data[features].copy()
y_val = val_data[target]

# Convert problematic object columns to float
for col in ['var_lag1_lag2', 'delta_rel_lag1_lag2']:
    if col in X_train.columns:
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    if col in X_val.columns:
        X_val[col] = pd.to_numeric(X_val[col], errors='coerce')


# 7. Entrenar modelo
mejores_params = {
    'n_estimators': 828,
    'learning_rate': 0.12429524985447538,
    'max_depth': 15,
    'num_leaves': 486,
    'min_data_in_leaf': 286,
    'min_child_weight': 0.5619194307398462,
    'subsample': 0.726053141459008,
    'subsample_freq': 7,
    'colsample_bytree': 0.7797069254086433,
    'colsample_bynode': 0.6383449194049172,
    'reg_alpha': 4.072667326486733,
    'reg_lambda': 0.34147791579771325,
    'max_bin': 495,
    'min_split_gain': 0.7124778300368457,
    'cat_smooth': 42.47362024156224,
    'random_state': 42,
    'boosting_type':  'dart',
    'verbosity': -1,
    'linear_tree': True
}

modelo = lgb.LGBMRegressor(**mejores_params)
modelo.fit(X_train, y_train)

# 8. Validación
y_pred_val = modelo.predict(X_val)
print("MAE en diciembre 2019:", mean_absolute_error(y_val, y_pred_val))

# 9. Predicción enero 2020
enero_df = df_prod[df_prod['periodo'] == '2019-12-01'].copy()
enero_df['periodo'] = pd.to_datetime('2020-01-01')
enero_df['lag_1'] = df_prod[df_prod['periodo'] == '2019-12-01']['tn'].values
enero_df['lag_2'] = df_prod[df_prod['periodo'] == '2019-11-01']['tn'].values
enero_df['lag_3'] = df_prod[df_prod['periodo'] == '2019-10-01']['tn'].values
enero_df['mes'] = 1
enero_df['var_lag1_lag2'] = enero_df['lag_1'] / enero_df['lag_2']
enero_df['delta_lag1_lag2'] = enero_df['lag_1'] - enero_df['lag_2']
enero_df['delta_rel_lag1_lag2'] = enero_df['delta_lag1_lag2'] / enero_df['lag_2']
enero_df['delta_ma12_lag1'] = enero_df['lag_1'] - enero_df['media_movil_12m']
enero_df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

# Convert problematic object columns to float
for col in ['var_lag1_lag2', 'delta_rel_lag1_lag2']:
    if col in enero_df.columns:
        enero_df[col] = pd.to_numeric(enero_df[col], errors='coerce')

enero_df['tn'] = modelo.predict(enero_df[features])
enero_df['tn'] = modelo.predict(enero_df[features])

# 10. Predicción febrero 2020
feb_df = pd.DataFrame()
feb_df['product_id'] = enero_df['product_id']

# Crear los features necesarios usando los valores de enero_df
feb_df['lag_1'] = enero_df['tn'].values  # enero 2020 predicho
feb_df['lag_2'] = enero_df['lag_1'].values  # diciembre 2019
feb_df['lag_3'] = enero_df['lag_2'].values  # noviembre 2019
feb_df['mes'] = 2  # febrero
feb_df['media_movil_12m'] = enero_df['media_movil_12m'].values

feb_df['var_lag1_lag2'] = feb_df['lag_1'] / feb_df['lag_2']
feb_df['delta_lag1_lag2'] = feb_df['lag_1'] - feb_df['lag_2']
feb_df['delta_rel_lag1_lag2'] = feb_df['delta_lag1_lag2'] / feb_df['lag_2']
feb_df['delta_ma12_lag1'] = feb_df['lag_1'] - feb_df['media_movil_12m']
feb_df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

# Convert problematic object columns to float
for col in ['var_lag1_lag2', 'delta_rel_lag1_lag2']:
    if col in feb_df.columns:
        feb_df[col] = pd.to_numeric(feb_df[col], errors='coerce')

# Merge con febrero histórico
feb_df = feb_df.merge(febrero_hist, on='product_id', how='left')
features_final = features + ['febrero_hist']

# 11. Predicción (solo con los features del modelo)
feb_df['tn_predicho'] = modelo.predict(feb_df[features])



# 12. Exportar resultados
resultado = feb_df[['product_id', 'tn_predicho']]
resultado.to_csv("prediccionfeb20top20nuevo.csv", index=False)
print("Predicción febrero 2020 guardada.")


MAE en diciembre 2019: 2.039626424874509
Predicción febrero 2020 guardada.


In [2]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# 1. Cargar y preparar datos
df = pd.read_csv("top20nuevo.csv")
df['periodo'] = pd.to_datetime(df['periodo'])

# 2. Agrupar por producto y mes
df_prod = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# 3. Crear features adicionales
# 3.1 Promedio histórico de febreros
febreros = df_prod[df_prod['periodo'].dt.month == 2]
febrero_hist = febreros[febreros['periodo'].dt.year < 2020].groupby('product_id')['tn'].mean().reset_index()
febrero_hist.rename(columns={'tn': 'febrero_hist'}, inplace=True)

# 3.2 Media móvil 12 meses
df_prod = df_prod.sort_values(['product_id', 'periodo'])
df_prod['media_movil_12m'] = (
    df_prod.groupby('product_id')['tn']
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)

# 4. Crear lags
def crear_lags(df, lags=[1, 2, 3]):
    df_lags = df.copy()
    for lag in lags:
        df_lags[f'lag_{lag}'] = df_lags.groupby('product_id')['tn'].shift(lag)
    return df_lags

df_lags = crear_lags(df_prod)
df_lags['mes'] = df_lags['periodo'].dt.month

# 5. Calcular deltas
df_lags['var_lag1_lag2'] = df_lags['lag_1'] / df_lags['lag_2']
df_lags['delta_lag1_lag2'] = df_lags['lag_1'] - df_lags['lag_2']
df_lags['delta_rel_lag1_lag2'] = df_lags['delta_lag1_lag2'] / df_lags['lag_2']
df_lags['delta_ma12_lag1'] = df_lags['lag_1'] - df_lags['media_movil_12m']
df_lags.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

# 6. Preparar entrenamiento y validación
df_train = df_lags.dropna()

features = [
    'lag_1', 'lag_2', 'lag_3', 'mes', 'media_movil_12m', 'var_lag1_lag2',
    'delta_lag1_lag2', 'delta_rel_lag1_lag2', 'delta_ma12_lag1'
]
target = 'tn'

train_data = df_train[df_train['periodo'] < '2019-12-01']
val_data = df_train[df_train['periodo'] == '2019-12-01']

X_train = train_data[features].copy()
y_train = train_data[target]
X_val = val_data[features].copy()
y_val = val_data[target]

# Convert problematic object columns to float
for col in ['var_lag1_lag2', 'delta_rel_lag1_lag2']:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    X_val[col] = pd.to_numeric(X_val[col], errors='coerce')

# 7. Entrenar modelo
mejores_params = {
    'n_estimators': 828,
    'learning_rate': 0.12429524985447538,
    'max_depth': 15,
    'num_leaves': 486,
    'min_data_in_leaf': 286,
    'min_child_weight': 0.5619194307398462,
    'subsample': 0.726053141459008,
    'subsample_freq': 7,
    'colsample_bytree': 0.7797069254086433,
    'colsample_bynode': 0.6383449194049172,
    'reg_alpha': 4.072667326486733,
    'reg_lambda': 0.34147791579771325,
    'max_bin': 495,
    'min_split_gain': 0.7124778300368457,
    'cat_smooth': 42.47362024156224,
    'random_state': 42,
    'boosting_type':  'dart',
    'verbosity': -1,
    'linear_tree': True
}

modelo = lgb.LGBMRegressor(**mejores_params)
modelo.fit(X_train, y_train)

# 8. Validación
y_pred_val = modelo.predict(X_val)
print("MAE en diciembre 2019:", mean_absolute_error(y_val, y_pred_val))

# 9. Predicción enero 2020
enero_df = df_prod[df_prod['periodo'] == '2019-12-01'].copy()
enero_df['periodo'] = pd.to_datetime('2020-01-01')
enero_df['lag_1'] = df_prod[df_prod['periodo'] == '2019-12-01']['tn'].values
enero_df['lag_2'] = df_prod[df_prod['periodo'] == '2019-11-01']['tn'].values
enero_df['lag_3'] = df_prod[df_prod['periodo'] == '2019-10-01']['tn'].values
enero_df['mes'] = 1
enero_df['media_movil_12m'] = df_prod[df_prod['periodo'] == '2019-12-01']['media_movil_12m'].values
enero_df['var_lag1_lag2'] = enero_df['lag_1'] / enero_df['lag_2']
enero_df['delta_lag1_lag2'] = enero_df['lag_1'] - enero_df['lag_2']
enero_df['delta_rel_lag1_lag2'] = enero_df['delta_lag1_lag2'] / enero_df['lag_2']
enero_df['delta_ma12_lag1'] = enero_df['lag_1'] - enero_df['media_movil_12m']
enero_df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

for col in ['var_lag1_lag2', 'delta_rel_lag1_lag2']:
    enero_df[col] = pd.to_numeric(enero_df[col], errors='coerce')

enero_df['tn'] = modelo.predict(enero_df[features])

# 10. Predicción febrero 2020
feb_df = pd.DataFrame()
feb_df['product_id'] = enero_df['product_id']
feb_df['lag_1'] = enero_df['tn'].values
feb_df['lag_2'] = enero_df['lag_1'].values
feb_df['lag_3'] = enero_df['lag_2'].values
feb_df['mes'] = 2
feb_df['media_movil_12m'] = enero_df['media_movil_12m'].values
feb_df['var_lag1_lag2'] = feb_df['lag_1'] / feb_df['lag_2']
feb_df['delta_lag1_lag2'] = feb_df['lag_1'] - feb_df['lag_2']
feb_df['delta_rel_lag1_lag2'] = feb_df['delta_lag1_lag2'] / feb_df['lag_2']
feb_df['delta_ma12_lag1'] = feb_df['lag_1'] - feb_df['media_movil_12m']
feb_df.replace([float('inf'), -float('inf')], pd.NA, inplace=True)

for col in ['var_lag1_lag2', 'delta_rel_lag1_lag2']:
    feb_df[col] = pd.to_numeric(feb_df[col], errors='coerce')

# 11. Merge con histórico y predicción
feb_df = feb_df.merge(febrero_hist, on='product_id', how='left')
feb_df['tn_predicho'] = modelo.predict(feb_df[features])
feb_df['tn_predicho_ajustada'] = 0.6 * feb_df['tn_predicho'] + 0.4 * feb_df['febrero_hist']

# 12. Exportar resultados
resultado = feb_df[['product_id', 'tn_predicho_ajustada']].copy()
resultado.rename(columns={'tn_predicho_ajustada': 'tn_predicho'}, inplace=True)
resultado.to_csv("prediccionfeb20top20nuevoJ.csv", index=False)
print("Predicción febrero 2020 ajustada guardada.")


MAE en diciembre 2019: 2.039626424874509
Predicción febrero 2020 ajustada guardada.
